In [1]:
import pandas as pd
import os

In [20]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error : Creating Directory'+directory)

In [26]:
def changePath(directory):
    currentPath = os.getcwd()
    chage_path = currentPath+directory
    try:
        if os.path.exists(directory[1:]):
            os.chdir(change_path)
    except OSError:
        print('Error : Changing Path'+directory)

In [28]:
if __name__=='__main__':
    print('\포함 입력하시오')
    data_folder_path=input()
changePath(data_folder_path)

\포함 입력하시오
/Bank


In [29]:
os.getcwd()

'C:\\Users\\gachon\\Waton\\RAWDATA'

각 Bank 별 폴더 생성
- 그 하위 폴더에 cell 값들 생성 

In [31]:
def CreateAdjCell(column_list):
    file_list=os.listdir(os.getcwd())
    file_count=len(file_list)
    createFolder('BANKS')
    current_path=os.getcwd()
    
    for i in range(file_count):
        file_path = file_list[i]
        delete_csv_path = file_path[:-4]
        df = pd.read_csv(file_path, encoding='utf8')
        maxcell = max(df['CellNo'])
        
        for j in range(1,maxcell+1):
            cell_index = df.index[df['CellNo']==j].tolist()
            df_cell = df.iloc[cell_index, :]
            new_cell_file = df_cell
            
            os.chdir(current_path+)
    

SyntaxError: invalid syntax (<ipython-input-31-fd6b884eb6a7>, line 18)

# BANK_TO_CELL
- 보름 이상 결측치는 구분

In [19]:
df=pd.read_csv('BANKS/m2_s2_bank_0053979e93612348a180e1133103725e__site_0d39d11b52565146a236a646d49133f4.csv')
df = df[['KeyTime','CellNo','ResistValue']]
df['KeyTime']=pd.to_datetime(df['KeyTime'],format="%Y/%m/%d")
for x in range(1,97):
    cell_1 = df[df['CellNo']==x].reset_index(drop=True)
    
    cut_date_value=15
    min_date=120
    cnt=1
    last=0
    for i in range(1,len(cell_1)):
        # 장기간 비어있는 cell, index 찾기
        if abs(cell_1['KeyTime'][i]-cell_1['KeyTime'][i-1]).days>cut_date_value:
            filename='cell_'+str(x)+'_'+str(cnt)+'.csv'
            data = cell_1[last:i].reset_index(drop=True)
            # Data의 측정 기간이 2달 이상인 경우
            if len(data)>120:
                data.to_csv(filename, index=False)
                last=i
                cnt+=1
            filename='cell_'+str(x)+'_'+str(cnt)+'.csv'
            data = cell_1[i:].reset_index(drop=True)
            if len(data)>120:
                data.to_csv(filename, index=False)
                last=i
                cnt+=1
            else: continue


# CELL_EDA

In [30]:
# 빈 날짜 결측치 채우기 (1일씩)
cell_1_1 = pd.read_csv('cell_1_1.csv')
chk_cell = cell_1_1

for i in range(1,(len(chk_cell)-1)):
    if abs(pd.Timestamp(chk_cell['KeyTime'][i]) - pd.Timestamp(chk_cell['KeyTime'][i-1])).days >= 1 :
        x= abs(pd.Timestamp(chk_cell['KeyTime'][i]) - pd.Timestamp(chk_cell['KeyTime'][i-1])).days
        # 비어있는 날짜 내부저항 -> 앞,뒤 평균값
        avg_resist = (chk_cell['ResistValue'][i-1]+chk_cell['ResistValue'][i])/2
        cnt=0
        while cnt<1:
            cnt+=1
            day=1
            idx=i
            new_data={'KeyTime': pd.Timestamp(chk_cell['KeyTime'][i-1])+pd.Timedelta('12H'),
                     'CellNo':chk_cell['CellNo'][i-1],
                     'ResistValue':avg_resist}
            tmp1=chk_cell[chk_cell.index<idx]
            tmp2=chk_cell[chk_cell.index>=idx]
            chk_cell = tmp1.append(new_data, ignore_index=True).append(tmp2,ignore_index=True)
            idx+=1
        #print(chk_cell[99:110])
chk_cell.to_csv('cell_1_1_adj.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'cell_1_1.csv'

In [ ]:
df = pd.read_csv('cell_1_1_adj.csv')
chk_cell = df

# 하루 2번 측정되었는지 확인
cnt_date=1
chk=pd.Timestamp(chk_cell['KeyTime'][0]).day
have_to_remove=[]
for i in range(1,len(chk_cell)):    
    # 하루 2번 정상적으로 측정되었는지 확인하기
    if chk == pd.Timestamp(chk_cell['KeyTime'][i]).day:
        cnt_date+=1
        if cnt_date>2:
            have_to_remove.append(pd.Timestamp(chk_cell['KeyTime'][i]))
            chk_cell.drop(i,axis=0,inplace=True)
    else:
        cnt_date=1
        chk=pd.Timestamp(chk_cell['KeyTime'][i]).day
chk_cell.to_csv('cell_1_1_adjv2.csv', index=False)

In [ ]:
def CreateCellCSVfile(column_list):
    file_list = os.listdir(os.getcwd())
    file_count = len(file_list)
    createFolder('BANK_NAME')
    current_path = os.getcwd()
    
    for i in range(file_count):
        file_path=file_list[i]
        delete_csv_path = file_path[:-4]
        df = pd.read_csv(file_path,encoding='utf8')
        maxcell = max(df['CellNo'])
        
        for j in range(maxcell):
            cell_index = df.index[df['CellNo']==(j+1)].tolist()
            df_cell = df.iloc[cell_index, :]
            new_cell_file = df_cell[column_list]
            
            os.chdir(current_path + '\cell')
            save_path = delete_csv_path + '_' + str(j+1).zfill(2) + '.csv'
            new_cell_file.to_csv(save_path, index=False, encoding='utf8')
            os.chdir(current_path)